In [37]:
import pandas as pd, urllib.request, json
from datetime import datetime

std_date_format = "%d/%m/%Y"

In [38]:
START_DATE_FINAL = '01/01/2000'
END_DATE_FINAL = '31/03/2021'

initial_day = START_DATE_FINAL.split('/')[0]
initial_month = START_DATE_FINAL.split('/')[1]
initial_year = START_DATE_FINAL.split('/')[2]

final_day = END_DATE_FINAL.split('/')[0]
final_month = END_DATE_FINAL.split('/')[1]
final_year = END_DATE_FINAL.split('/')[2]

In [39]:

selic_rates_url = f'https://api.bcb.gov.br/dados/serie/bcdata.sgs.11/dados?formato=json&dataInicial={initial_day}/{initial_month}/{initial_year}&dataFinal={final_day}/{final_month}/{final_year}'

with urllib.request.urlopen(selic_rates_url) as url:
    data = json.loads(url.read().decode())
    selic_rates = pd.DataFrame(data)

selic_rates['data'] = pd.to_datetime(selic_rates['data'], format=std_date_format)
selic_rates = selic_rates.loc[(selic_rates['data'].between(START_DATE_FINAL, END_DATE_FINAL))]
selic_rates['valor'] = pd.to_numeric(selic_rates['valor'])

c:\Users\Teka\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\arraylike.py:52: UserWarning: Parsing '31/03/2021' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  return self._cmp_method(other, operator.le)


In [40]:
sum_df = selic_rates['valor'].rolling(500).sum()
idxmax = sum_df.idxmax()

In [41]:
best_period = selic_rates[selic_rates.index.isin(range(idxmax, idxmax + 501))]
best_period

,data,valor
957,2003-10-22,0.071813
958,2003-10-23,0.068552
959,2003-10-24,0.068552
960,2003-10-27,0.068552
961,2003-10-28,0.068552
...,...,...
1453,2005-10-10,0.070552
1454,2005-10-11,0.070552
1455,2005-10-13,0.070552
1456,2005-10-14,0.070552


In [42]:
initial_best_date = best_period['data'].dt.strftime(std_date_format).values[0]
final_best_date = best_period['data'].dt.strftime(std_date_format).values[-1]

rates_list = best_period['valor'].tolist()
total_rate = 1

for r in rates_list:
    total_rate *= (1 + r)
    
print(total_rate)

36387732955249.01
